In [25]:
Packages <- c("dplyr",  "RMySQL",  "tidyr", "lubridate", "stringr")
invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))
setwd('/local/home/katrinac/parentage')

load("~/parentage/r_data/sampled_area_each_year.RData")
load("~/parentage/r_data/anems_visited_by_year.RData")
load("~/parentage/r_data/total_sampling_across_years.RData")

#download.file(url = "https://raw.githubusercontent.com/pinskylab/genomics/master/data/known_issues.csv", destfile = "~/parentage/r_data/known_issues.csv")
issues <- read.csv("~/parentage/r_data/known_issues.csv", header=T, stringsAsFactors = F)
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/fish_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/fish_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/anem_db.RData?raw=true", destfile = "~/parentage/r_data/anem_db.RData")
load("~/parentage/r_data/anem_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/dives_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/dives_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/fish_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/fish_db.RData")
load("~/parentage/r_data/gps_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/anems_tagged.RData", destfile = "~/parentage/r_data/anems_tagged.RData")
load("~/parentage/r_data/anems_tagged.RData")
#download.file(url = "https://github.com/pinskylab/genomics/blob/master/data/fish-obs.RData?raw=true", destfile = "~/parentage/r_data/fish-obs.RData")
fish_obs <- readRDS("~/parentage/r_data/fish-obs.RData") 
#download.file(url = "https://raw.githubusercontent.com/pinskylab/db_backups/master/ligation_1-7-20.csv", destfile = "~/parentage/r_data/ligation_db.csv")
lig_db <- read.csv("~/parentage/r_data/ligation_db.csv", header=T, stringsAsFactors = F)
#download.file(url = "https://raw.githubusercontent.com/pinskylab/db_backups/master/digest_1-7-20.csv", destfile = "~/parentage/r_data/digest_db.csv")
dig_db <- read.csv("~/parentage/r_data/digest_db.csv", header=T, stringsAsFactors = F)
#download.file(url = "https://raw.githubusercontent.com/pinskylab/db_backups/master/extraction_1-7-20.csv", destfile = "~/parentage/r_data/extraction_db.csv")
ext_db <- read.csv("~/parentage/r_data/extraction_db.csv", header=T, stringsAsFactors = F)
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0


In [2]:
dat_gen <- read.table("/local/home/katrinac/parentage/colony2/20190523_1340loci/input/20190523_1340_loci_seq33-03_noLD.txt", colClasses= "character",  quote="", header=TRUE, row.names=NULL)
results_dist <- read.csv(file="~/parentage/colony2/20200605_1340loci/results/parentage_results_allyears.csv", header=T)


In [26]:
#from Allison, just putting all the meta data together (Constants_database_common_functions.R)
##### Match up other relevant info (site, date, fish_indiv, etc.) to fish in the clownfish table
# Pull out year and month into a separate column in dives_db
dives_db_processed <- dives_db %>%
  mutate(year = as.integer(substring(date,1,4))) %>%
  mutate(month = as.integer(substring(date,6,7))) %>%
  mutate(dive_date = date(date))

# Pull all APCL caught or otherwise in the clownfish table
allfish_fish <- fish_db %>%
  select(fish_table_id, anem_table_id, fish_spp, sample_id, anem_table_id, recap, tag_id, color, sex, size, fish_obs_time, fish_notes) %>%
  filter(fish_spp == 'APCL') %>%
  mutate(size = as.numeric(size))  # make the size numeric (rather than chr) so can do means and such

# and their corresponding anemones
allfish_anems <- anem_db %>%
  select(anem_table_id, dive_table_id, anem_obs, anem_obs_time, anem_id, old_anem_id, anem_spp, anem_dia) %>%
  filter(anem_table_id %in% allfish_fish$anem_table_id)

# and the corresponding dive info
allfish_dives <- dives_db_processed %>%
  select(dive_table_id, dive_type, date, year, month, site, gps, dive_notes) %>%
  filter(dive_table_id %in% allfish_anems$dive_table_id) 

# join together
allfish_caught <- left_join(allfish_fish, allfish_anems, by="anem_table_id")
allfish_caught <- left_join(allfish_caught, allfish_dives, by="dive_table_id")

# add in the gen_ids and fish_indiv (now in a separate gen_id table) - gen_id only comes in the time the fish was sequenced, not at all captures
allfish_caught <- left_join(allfish_caught, (fish_obs %>% select(fish_table_id, gen_id, fish_indiv)), by = "fish_table_id")

#get sample ids for ligation ids
ext_db <- ext_db %>%
    select(extraction_id, sample_id)
dig_db <- dig_db %>%
    select(extraction_id, digest_id)
lig_db <- lig_db %>%
    select(ligation_id, digest_id)
ext_dig <- left_join(ext_db, dig_db, by="extraction_id")
dig_lig <- left_join(ext_dig, lig_db, by="digest_id") %>% 
    filter(!is.na(ligation_id))
lig_samp <- dig_lig %>%
    select(ligation_id, sample_id) 


fish_in_gen_lig <- dat_gen %>% #pull of the ligation_ids of fish in dat_gen, and remove those in the issues table
    select(ligation_id) %>%
    filter(ligation_id %!in% issues$ligation_id) #5 fish in issues table, these are dropped
nrow(dat_gen) -nrow(fish_in_gen_lig) 

fish_in_gen_samp <- lig_samp %>% 
    filter(ligation_id %in%dat_gen$ligation_id) %>%
    distinct(ligation_id, sample_id, .keep_all = T) #all of the ligation_ids in fish_in_gen_samp are in fish_in_gen_lig. Are the extra 7 sample_ids just multiples for regenotyped fish? YES see the following when this line is commented out: test %>% group_by(ligation_id, sample_id) %>% summarise(n=n()) %>% filter(n >1)

nrow(fish_in_gen_lig)- nrow(fish_in_gen_samp) #should be 0

sampled_fish <- left_join(allfish_caught, lig_samp,  by="sample_id") %>%
    mutate(time_date =as.character(str_c(date, anem_obs_time, sep = " "))) %>%
    mutate(time_date = ymd_hms(time_date))%>%
    mutate(time_date = force_tz(time_date, tzone = "Asia/Manila")) %>%
    mutate(time_date = with_tz(time_date, tzone = "UTC")) %>%
    mutate(year = year(time_date)) %>%
    mutate(month = month(time_date)) %>%
    mutate(day = day(time_date)) %>%
    mutate(hour = hour(time_date)) %>%
    mutate(minute = minute(time_date)) 
    

gps <- gps_db %>%
    mutate(lat=as.numeric(lat)) %>%
    mutate(lon=as.numeric(lon)) %>%
    mutate(time_date = ymd_hms(time)) %>%
    mutate(year = year(time_date)) %>%
    mutate(month = month(time_date)) %>%
    mutate(day = day(time_date)) %>%
    mutate(hour = hour(time_date)) %>%
    mutate(minute = minute(time_date)) %>%
    select(-time_date, -elev, -time) %>%
    group_by(unit, year, month, day, hour, minute) %>%
    mutate(lat =mean(lat)) %>% #create within minute averages
    mutate(lon=mean(lon)) %>%
    distinct(.keep_all = T)
#*added
#the long haul of adding lat lons
#results_meta <- par_res %>%
#    mutate(time_date =as.character(str_c(date, anem_obs_time, sep = " "))) %>%
#    mutate(time_date = ymd_hms(time_date))%>%
#    mutate(time_date = force_tz(time_date, tzone = "Asia/Manila")) %>%
#    mutate(time_date = with_tz(time_date, tzone = "UTC")) %>%
#    mutate(year = year(time_date)) %>%
#    mutate(month = month(time_date)) %>%
#    mutate(day = day(time_date)) %>%
#    mutate(hour = hour(time_date)) %>%
#    mutate(minute = minute(time_date)) %>%
#    mutate(par_time_date =as.character(str_c(par_date, par_anem_obs_time, sep = " "))) %>%
#    mutate(par_time_date = ymd_hms(par_time_date))%>%
#    mutate(par_time_date = force_tz(par_time_date, tzone = "Asia/Manila")) %>%
#    mutate(par_time_date = with_tz(par_time_date, tzone = "UTC")) %>%
#    mutate(par_year = year(par_time_date)) %>%
#    mutate(par_month = month(par_time_date)) %>%
#    mutate(par_day = day(par_time_date)) %>%
#    mutate(par_hour = hour(par_time_date)) %>%
#    mutate(par_minute = minute(par_time_date))
#
#gps <- gps_db %>%
#    mutate(lat=as.numeric(lat)) %>%
#    mutate(lon=as.numeric(lon)) %>%
#    mutate(time_date = ymd_hms(time)) %>%
#    mutate(year = year(time_date)) %>%
#    mutate(month = month(time_date)) %>%
#    mutate(day = day(time_date)) %>%
#    mutate(hour = hour(time_date)) %>%
#    mutate(minute = minute(time_date)) %>%
#    select(-time_date, -elev, -time) %>%
#    group_by(unit, year, month, day, hour, minute) %>%
#    mutate(lat =mean(lat)) %>% #create within minute averages
#    mutate(lon=mean(lon)) %>%
#    distinct(.keep_all = T)
##*added

fish_meta <- sampled_fish %>%#join with the rest of the data
     select(fish_indiv,ligation_id, sample_id, size, color, sex, anem_id, anem_spp, anem_dia, date, month, site)
###join with genetic data
dat_gen_meta <-  left_join(dat_gen, fish_meta, by="ligation_id") %>% 
    filter(!is.na(fish_indiv)) %>%
    select(fish_indiv,ligation_id, sample_id, size, color, sex, anem_id, anem_spp, anem_dia, date, month, site, everything())

[1] 0

[1] 0

Warning message:
“Problem with `mutate()` input `time_date`.
ℹ  2 failed to parse.
ℹ Input `time_date` is `ymd_hms(time_date)`.”Warning message:
“ 2 failed to parse.”

In [18]:
str(allfish_caught)

tibble [7,715 × 27] (S3: tbl_df/tbl/data.frame)
 $ fish_table_id: num [1:7715] 7683 487 31 2510 6057 ...
 $ anem_table_id: num [1:7715] 7385 399 15 2130 6473 ...
 $ fish_spp     : chr [1:7715] "APCL" "APCL" "APCL" "APCL" ...
 $ sample_id    : chr [1:7715] "APCL16_337" "APCL12_236" "APCL12_019" "APCL14_245" ...
 $ recap        : chr [1:7715] NA NA NA NA ...
 $ tag_id       : chr [1:7715] NA NA NA NA ...
 $ color        : chr [1:7715] "YR" "W" NA "Y" ...
 $ sex          : chr [1:7715] "J" "J" NA "J" ...
 $ size         : num [1:7715] 3.5 3.5 3.1 5.5 3.5 3.4 4.8 8.8 4 4.1 ...
 $ fish_obs_time: chr [1:7715] NA NA NA NA ...
 $ fish_notes   : chr [1:7715] NA NA NA NA ...
 $ dive_table_id: num [1:7715] 318 19 3 82 259 59 89 91 36 95 ...
 $ anem_obs     : num [1:7715] 1786 NA NA 534 152 ...
 $ anem_obs_time: chr [1:7715] "12:02:00" "11:51:00" "11:44:00" "15:39:00" ...
 $ anem_id      : num [1:7715] 2407 NA NA 524 1040 ...
 $ old_anem_id  : chr [1:7715] NA NA NA NA ...
 $ anem_spp     : chr [1:

In [29]:
full_meta <- left_join(sampled_fish, gps, by=c(gps="unit", "year", "month", "day", "hour", "minute")) %>% select(-dive_notes, -year, -month, -day, -hour, -minute)


In [30]:
sum(!is.na(full_meta$lat)==T)

[1] 8471

In [31]:
#write.csv(full_meta, file="~/parentage/r_data/FullFishMetaDataForMolEco.csv", quote=F, row.names=F )
#write.csv(full_meta, file="~/DispersalVariation/data/FullFishMetaDataForMolEco.csv", quote=F, row.names=F )

In [6]:
dat_gen <- read.table("/local/home/katrinac/parentage/colony2/20190523_1340loci/input/20190523_1340_loci_seq33-03_noLD.txt", colClasses= "character",  quote="", header=TRUE, row.names=NULL)
head(dat_gen)

ligation_id,dDocent_Contig_11_59.01,dDocent_Contig_11_59.02,dDocent_Contig_11_87.01,dDocent_Contig_11_87.02,dDocent_Contig_11_93.01,dDocent_Contig_11_93.02,dDocent_Contig_11_117.01,dDocent_Contig_11_117.02,dDocent_Contig_11_126.01,...,dDocent_Contig_207387_131.01,dDocent_Contig_207387_131.02,dDocent_Contig_207875_52.01,dDocent_Contig_207875_52.02,dDocent_Contig_226526_58.01,dDocent_Contig_226526_58.02,dDocent_Contig_233984_13.01,dDocent_Contig_233984_13.02,dDocent_Contig_233984_104.01,dDocent_Contig_233984_104.02
L0255,01,02,01,01,01,01,01,01,01,...,01,02,01,02,02,02,01,02,01,02
L0256,01,02,01,01,01,01,01,01,01,...,02,02,01,02,02,02,01,02,01,02
L0257,01,02,01,01,01,01,01,01,01,...,01,01,02,02,02,02,02,02,02,02
L0258,01,01,01,01,02,01,01,01,01,...,01,02,01,02,02,02,01,02,01,02
L0259,01,01,01,01,01,02,01,01,01,...,02,02,02,02,02,02,01,02,01,02
L0260,01,01,01,01,01,01,01,01,01,...,01,02,01,02,01,02,01,02,01,02


In [7]:
results_dist <- read.csv(file="~/parentage/colony2/20200605_1340loci/results/parentage_results_allyears.csv", header=T)
head(results_dist)

par_fish_indiv,par_size,par_color,par_sex,par_gen_id,par_ligation_id,par_sample_id,par_site,par_date,par_anem_obs_time,...,par_year,par_month,par_day,par_hour,par_minute,offs_lat,offs_lon,par_lat,par_lon,dist_par_km
25,10.4,YP,F,25,L2238,APCL13_510,Visca,2013-06-01,17:18:00,...,2013,6,1,9,18,10.74343,124.7869,10.74427,124.7865,0.1049982
124,7.5,NA,NA,124,L1770,APCL12_110,Wangag,2012-05-09,15:05:00,...,2012,5,9,7,5,10.87312,124.7121,10.87166,124.7140,0.2673390
132,8.0,NA,NA,132,L1769,APCL12_109,Wangag,2012-05-09,15:09:00,...,2012,5,9,7,9,10.87312,124.7121,10.87165,124.7141,0.2708801
133,9.0,O,M,133,L2217,APCL12_117,Wangag,2012-05-09,15:00:00,...,2012,5,9,7,0,10.87312,124.7121,10.87168,124.7140,0.2618357
133,9.0,O,M,133,L2217,APCL12_117,Wangag,2012-05-09,15:00:00,...,2012,5,9,7,0,10.86902,124.7165,10.87168,124.7140,0.4013014
146,10.4,NA,NA,146,L2345,APCL12_030,Visca,2012-05-06,15:31:00,...,2012,5,6,7,31,10.87427,124.7107,10.74399,124.7863,16.6768523


In [8]:
options(dplyr.summarise.inform = FALSE)

In [9]:
nrow(dat_gen_meta %>% distinct(fish_indiv))

[1] 2405

In [10]:
dat_gen_meta %>%
    group_by(anem_spp) %>%
    summarise(NObs=n())

anem_spp,NObs
????,35
EMPT,2
ENQD,154
HEAR,47
HECR,1789
HEMA,7
MADO,278
STHD,14
STME,80


In [11]:
AnemSppLigIDs <- fish_meta %>% #this will pull out all ligation ids associated with a fish
    filter(!is.na(fish_indiv)) %>% 
    filter(fish_indiv %in% dat_gen_meta$fish_indiv)%>%
    filter(ligation_id %!in% issues$ligation_id) %>% 
    filter(!is.na(ligation_id)) %>% 
    select(fish_indiv, ligation_id, anem_spp) %>% 
    filter(anem_spp=="HECR" | anem_spp=="STME") %>% 
    mutate(anem_spp_binary=ifelse(anem_spp=="HECR", 1, 2)) %>% 
    mutate(sampligation_id=paste("APCL", ligation_id, sep="_"))

AnemSppLigIDs %>% distinct(fish_indiv, .keep_all = T) %>% group_by(anem_spp_binary) %>% summarise(NObs=n())

anem_spp_binary,NObs
1,1815
2,81


In [65]:
#write.table(AnemSppLigIDs, file="~/ClownfishGWAS/script_output/ForMolEcoGroupAnemSppLigIDs.txt", sep="\t")

In [13]:
write.csv(AnemSppLigIDs, file="~/ClownfishGWAS/script_output/ForMolEcoGroupAnemSppLigIDs.csv", row.names = F, col.names = T, quote=F)

Warning message in write.csv(AnemSppLigIDs, file = "~/ClownfishGWAS/script_output/ForMolEcoGroupAnemSppLigIDs.csv", :
“attempt to set 'col.names' ignored”

In [79]:
nrow(fish_meta %>% 
    filter(!is.na(fish_indiv)) %>% 
    filter(fish_indiv %in% dat_gen_meta$fish_indiv)%>%
    filter(ligation_id %!in% issues$ligation_id) %>% 
    filter(!is.na(ligation_id)))

[1] 3054

In [31]:
#now read in the plink ped file and attach phenotypes
ped <-read.table("~/ClownfishGWAS/script_output/ForMolEcoGroup.recode.ped", sep="\t", header=F)
head(ped)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V2003,V2004,V2005,V2006,V2007,V2008,V2009,V2010,V2011,V2012
APCL_L0255,APCL_L0255,0,0,0,1,C,T,A,G,...,C,A,T,A,C,G,A,G,G,A
APCL_L0256,APCL_L0256,0,0,0,1,C,T,A,G,...,C,A,T,A,C,G,A,G,G,A
APCL_L0257,APCL_L0257,0,0,0,1,C,T,A,G,...,A,A,A,A,G,G,G,G,A,A
APCL_L0258,APCL_L0258,0,0,0,1,C,C,A,A,...,C,A,T,A,C,G,A,G,G,A
APCL_L0259,APCL_L0259,0,0,0,1,C,C,A,G,...,C,A,T,A,C,G,A,G,G,A
APCL_L0260,APCL_L0260,0,0,0,1,C,C,G,G,...,C,A,T,A,C,G,A,G,G,A


In [32]:
#pull out just the sampligation_id and anem phenotype and then order like what's in the ped file
test <- (AnemSppLigIDs %>% select(sampligation_id, anem_spp_binary) %>% arrange(sampligation_id))
ordered_phenotypes <- left_join(ped %>% select(1) %>% rename(sampligation_id="V1"), test)
#then substitute the anem phenotype column in for the 6th column of the ped file, which is where the phenotype goes
ped[,6] <- ordered_phenotypes$anem_spp_binary
head(ped)

Joining, by = "sampligation_id"


V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V2003,V2004,V2005,V2006,V2007,V2008,V2009,V2010,V2011,V2012
APCL_L0255,APCL_L0255,0,0,0,1,C,T,A,G,...,C,A,T,A,C,G,A,G,G,A
APCL_L0256,APCL_L0256,0,0,0,1,C,T,A,G,...,C,A,T,A,C,G,A,G,G,A
APCL_L0257,APCL_L0257,0,0,0,1,C,T,A,G,...,A,A,A,A,G,G,G,G,A,A
APCL_L0258,APCL_L0258,0,0,0,1,C,C,A,A,...,C,A,T,A,C,G,A,G,G,A
APCL_L0259,APCL_L0259,0,0,0,1,C,C,A,G,...,C,A,T,A,C,G,A,G,G,A
APCL_L0260,APCL_L0260,0,0,0,1,C,C,G,G,...,C,A,T,A,C,G,A,G,G,A


In [33]:
#now write out for plink analysis
write.table(ped, file="~/ClownfishGWAS/script_output/ForMolEcoGroup.recode.ped", row.names = F, col.names = F, sep="\t", quote=F)

In [ ]:
#hmmm. I'm now realizing that we don't want any related individuals messing up the association study so they need to be removed, duh.